In [1]:
#!/usr/bin/env python3

import os
import cv2
import darknet
import json
import joblib

darknet.set_gpu(0)
net = darknet.load_net(b"cfg/yolov3.cfg", b"weights/yolov3.weights", 0)
meta = darknet.load_meta(b"cfg/coco.data")

cwd = os.getcwd()
print(cwd)
#print("Running ", __file__, " in ", cwd, ".")

prediction_data_path = "predictions.txt"

root = cwd + "/data/tdata/"

/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project


In [2]:
import re
files = os.listdir()
pattern = "(?P<file_name>(?P<base_name>[^_]*_(?P<datetime>(?P<date>\d*)_(?P<time>\d*)))\.pkl$)"
reg = re.compile(pattern)


matches = []

for file in sorted(files):
    match = reg.search(file)
    
    if match is not None:
        matches.append(match)

        
pickled_images = []
for match in matches:
    pickled_images.append(match.groupdict()["base_name"] + ".jpg")
    

    
print(pickled_images)

[]


In [3]:
data_files = os.listdir("data/tdata")

pattern_jpg = "(?P<file_name>(?P<base_name>[^_]*_(?P<datetime>(?P<date>\d*)_(?P<time>\d*)))\.jpg$)"
reg_jpg = re.compile(pattern_jpg)

matches_jpg = []

for data_file in data_files:
    match_jpg = reg_jpg.search(data_file)

    if match_jpg is not None:
        matches_jpg.append(match_jpg)
    
all_images = []
for match_jpg in matches_jpg:
    all_images.append(match_jpg.groupdict()["file_name"])
    
print(all_images)



['capture_20181130_005347.jpg', 'capture_20181130_005757.jpg', 'capture_20181130_005227.jpg', 'capture_20181130_005408.jpg', 'capture_20181130_005146.jpg', 'capture_20181130_005608.jpg', 'capture_20181130_005807.jpg', 'capture_20181126_181828.jpg', 'capture_20181130_004701.jpg', 'capture_20181130_005156.jpg', 'capture_20181130_005618.jpg', 'capture_20181130_004712.jpg', 'capture_20181130_005418.jpg', 'capture_20181130_005237.jpg', 'capture_20181130_005357.jpg', 'capture_20181130_005708.jpg', 'capture_20181130_005327.jpg', 'capture_20181130_005247.jpg', 'capture_20181130_005508.jpg', 'capture_20181130_005124.jpg', 'capture_20181130_005518.jpg', 'capture_20181130_005257.jpg', 'capture_20181130_005135.jpg', 'capture_20181130_005337.jpg', 'capture_20181130_005718.jpg', 'capture_20181130_005728.jpg', 'capture_20181130_005648.jpg', 'capture_20181130_005448.jpg', 'capture_20181130_005528.jpg', 'capture_20181130_005307.jpg', 'capture_20181130_005317.jpg', 'capture_20181130_005538.jpg', 'captur

In [4]:
unpickled_images = sorted(set(all_images)-set(pickled_images))
print(unpickled_images)

['capture_20181126_181825.jpg', 'capture_20181126_181826.jpg', 'capture_20181126_181827.jpg', 'capture_20181126_181828.jpg', 'capture_20181130_004651.jpg', 'capture_20181130_004701.jpg', 'capture_20181130_004712.jpg', 'capture_20181130_005124.jpg', 'capture_20181130_005135.jpg', 'capture_20181130_005146.jpg', 'capture_20181130_005156.jpg', 'capture_20181130_005206.jpg', 'capture_20181130_005217.jpg', 'capture_20181130_005227.jpg', 'capture_20181130_005237.jpg', 'capture_20181130_005247.jpg', 'capture_20181130_005257.jpg', 'capture_20181130_005307.jpg', 'capture_20181130_005317.jpg', 'capture_20181130_005327.jpg', 'capture_20181130_005337.jpg', 'capture_20181130_005347.jpg', 'capture_20181130_005357.jpg', 'capture_20181130_005408.jpg', 'capture_20181130_005418.jpg', 'capture_20181130_005428.jpg', 'capture_20181130_005438.jpg', 'capture_20181130_005448.jpg', 'capture_20181130_005458.jpg', 'capture_20181130_005508.jpg', 'capture_20181130_005518.jpg', 'capture_20181130_005528.jpg', 'captur

In [5]:
result_list = []

for file in unpickled_images:

    # Basename of the image file without the extension
    basename = os.path.splitext(file)[0]

    # Image file
    imagefile = os.path.join(root, file)

    # output text file -> trying to output results in json format
    outfile = basename + "_out.json"

    # output image name -> darkflow outputs to a png file.
    outimage = basename + "_out.png"

    # Compose the command to be executed - revise later
    # command = "./darknet detect cfg/yolov3.cfg ./yolov3.weights " + \
    #          imagefile + " >> " + prediction_data_path

    # Make an external command - This is where all the problems begin.
    #os.system(command)
    print(imagefile)
    r = darknet.detect(net, meta, imagefile.encode())
    joblib.dump(r, basename + ".pkl")
    result_list.append(r)

    """
    for i in len(r):
        tup = r[i]
        object_type = tup(0)
        prob = tup(1)
        coord = tup(2)
    """    


    # Challenges performing segmentation. Not quite sure how to draw \
    # the bounding boxes on the images. Trying to used opencv for this.
    """
    img = cv2.imread(imagefile)
    #for (cls_name, likelihood, (x, y, w, h)) in r:
        # cv2.putText(
        #     img,
        #     cls_name,
        #     (int(x + w/2), int(y + h/2)),
        #     cv2.FONT_HERSHEY_SIMPLEX,
        #     1,
        #     (255, 255, 255)
        # )
        # cv2.rectangle(
        #     img,
        #     (int(x), int(y)),
        #     (int(x + w), int(y + h)),
        #     (255, 255, 255)
        # )

    # Trying to force terminal output in json format
 
    with open(outfile, "w+") as f:
        json.dump([
           {
                "class": cls_name.decode(),
                "likelihood": likelihood
            } for (cls_name, likelihood, _) in r
        ], f)
 

    # cv2.imwrite(outimage, img)
    #cv2.imshow(imagefile, img)

    #tmp_img_path = os.path.join(cwd, "predictions.png")
    #out_img_path = os.path.join(cwd, "predictions", outimage)

    #os.rename(tmp_img_path, out_img_path)

"""



/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181126_181825.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181126_181826.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181126_181827.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181126_181828.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181130_004651.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181130_004701.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181130_004712.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_20181130_005124.jpg
/Users/ndapewaonyothi/Documents/DataScienceRetreat/DSR-2018/Final_Project/data/tdata/capture_201

In [6]:
print(result_list)
#max(result_list, key=lambda x: x[1])
# print(result_list[0:3])
#print(result_list[3][14])

[[(b'person', 0.9574810862541199, (443.546630859375, 337.1438293457031, 30.6710205078125, 99.24769592285156)), (b'person', 0.9430323243141174, (212.923828125, 372.9030456542969, 36.58625030517578, 91.71443939208984)), (b'person', 0.8935564160346985, (577.0828857421875, 313.9469909667969, 23.23914337158203, 55.083675384521484)), (b'person', 0.8814886808395386, (442.23980712890625, 608.315673828125, 69.89932250976562, 206.338623046875)), (b'person', 0.8539845943450928, (550.5533447265625, 312.5330810546875, 22.915437698364258, 61.22237014770508)), (b'truck', 0.8300362825393677, (118.01609802246094, 287.0903015136719, 183.07875061035156, 75.066162109375)), (b'person', 0.8235337138175964, (528.5616455078125, 633.2394409179688, 90.59176635742188, 185.0718231201172)), (b'person', 0.7925055027008057, (279.8171691894531, 345.8374938964844, 29.769866943359375, 89.94104766845703)), (b'person', 0.7922622561454773, (21.122053146362305, 313.40106201171875, 32.563377380371094, 56.8663444519043)), (b

In [7]:
#Why are you not working?
object_names = []
#object_prob = []

for object_list in result_list:
    for obj in object_list:
        if isinstance(obj,tuple):
            if len(obj) == 3: #Validate that the object has three variables
                name = obj[0]
                #prob = obj[1]
                
                if not (name in object_names): 
                    object_names.append(name)
               # if not (prob in object_prob):
               #     object_prob.append(prob)
                    
#There seem to be a corrupt file        
print(object_names) 

[b'person', b'truck', b'car', b'suitcase', b'handbag', b'umbrella', b'traffic light', b'backpack', b'tie']


In [8]:
print(result_list)

[[(b'person', 0.9574810862541199, (443.546630859375, 337.1438293457031, 30.6710205078125, 99.24769592285156)), (b'person', 0.9430323243141174, (212.923828125, 372.9030456542969, 36.58625030517578, 91.71443939208984)), (b'person', 0.8935564160346985, (577.0828857421875, 313.9469909667969, 23.23914337158203, 55.083675384521484)), (b'person', 0.8814886808395386, (442.23980712890625, 608.315673828125, 69.89932250976562, 206.338623046875)), (b'person', 0.8539845943450928, (550.5533447265625, 312.5330810546875, 22.915437698364258, 61.22237014770508)), (b'truck', 0.8300362825393677, (118.01609802246094, 287.0903015136719, 183.07875061035156, 75.066162109375)), (b'person', 0.8235337138175964, (528.5616455078125, 633.2394409179688, 90.59176635742188, 185.0718231201172)), (b'person', 0.7925055027008057, (279.8171691894531, 345.8374938964844, 29.769866943359375, 89.94104766845703)), (b'person', 0.7922622561454773, (21.122053146362305, 313.40106201171875, 32.563377380371094, 56.8663444519043)), (b

In [19]:
print("These are the detected object Names:\n ",object_names)

number_dict_total = {}
for name in object_names:
    number_dict_total[name] = 0

    
number_dicts_images = []

# result_list: list of length 4; each element is a list of detected_objects

counter = 0
image_counter = 0

for i, detected_object_list in enumerate(result_list):
        
    number_dict_image = {}
    for name in object_names:
        number_dict_image[name] = 0
 
    for tup in detected_object_list:
        if isinstance(tup,tuple):
            if len(tup) == 3:
                name = tup[0]
                if name in object_names:
                    number_dict_image[name] +=1
                    number_dict_total[name] +=1
            
      
    number_dicts_images.append(number_dict_image)
    
print("\nThese are the totals of the detected objects:")
number_dict_total

These are the detected object Names:
  [b'person', b'truck', b'car', b'suitcase', b'handbag', b'umbrella', b'traffic light', b'backpack', b'tie']

These are the totals of the detected objects:


{b'person': 982,
 b'truck': 6,
 b'car': 17,
 b'suitcase': 5,
 b'handbag': 9,
 b'umbrella': 3,
 b'traffic light': 1,
 b'backpack': 5,
 b'tie': 1}

In [22]:
#Print the total number of objects detected per image

print("t",number_dict_total)
print("---------------------------------------------------------------------")
for i in range(len(number_dicts_images)):
    print(i, number_dicts_images[i])

t {b'person': 982, b'truck': 6, b'car': 17, b'suitcase': 5, b'handbag': 9, b'umbrella': 3, b'traffic light': 1, b'backpack': 5, b'tie': 1}
---------------------------------------------------------------------
0 {b'person': 14, b'truck': 1, b'car': 3, b'suitcase': 1, b'handbag': 1, b'umbrella': 0, b'traffic light': 0, b'backpack': 0, b'tie': 0}
1 {b'person': 14, b'truck': 1, b'car': 1, b'suitcase': 0, b'handbag': 0, b'umbrella': 0, b'traffic light': 0, b'backpack': 0, b'tie': 0}
2 {b'person': 13, b'truck': 2, b'car': 0, b'suitcase': 0, b'handbag': 0, b'umbrella': 1, b'traffic light': 0, b'backpack': 0, b'tie': 0}
3 {b'person': 16, b'truck': 1, b'car': 2, b'suitcase': 0, b'handbag': 0, b'umbrella': 2, b'traffic light': 1, b'backpack': 0, b'tie': 0}
4 {b'person': 28, b'truck': 0, b'car': 0, b'suitcase': 1, b'handbag': 0, b'umbrella': 0, b'traffic light': 0, b'backpack': 0, b'tie': 0}
5 {b'person': 27, b'truck': 0, b'car': 0, b'suitcase': 0, b'handbag': 1, b'umbrella': 0, b'traffic light':

In [ ]:
# Try to create atleast one plot from Dictionary data
# Alternatively, try to feed some of the dictionary data into a pandas DF to create visuals. 


In [23]:
#Try and fix this - Window does not show image. 
for i in range(100000):
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow(imagefile, img)
cv2.destroyAllWindows()

NameError: name 'img' is not defined

In [25]:
# Try to plot, but then results are nested lists. 
"""
import matplotlib.pyplot as plt
imlens = []
for i,_im in enumerate(result_list):
    imlens.append(len(_im))
    timefilteredForce = plt.plot(result_list)
    timefilteredForce = pyplot.xlabel('points')
    timefilteredForce = pyplot.ylabel('Force')

plt.show()
imlens
"""


"\nimport matplotlib.pyplot as plt\nimlens = []\nfor i,_im in enumerate(result_list):\n    imlens.append(len(_im))\n    timefilteredForce = plt.plot(result_list)\n    timefilteredForce = pyplot.xlabel('points')\n    timefilteredForce = pyplot.ylabel('Force')\n\nplt.show()\nimlens\n"